# 아가모토의 눈
2학년 2학기 교내대회 작품용 코드  
mediapipe와 openCV를 활용하여 포즈를 감지하고 정해진 포즈를 취하면  
아가모토의 눈이 열리게 되는 작품

하드웨어 : 라즈베리파이, 아두이노, 모터, LED 등  
소프트웨어 : 파이썬(mediapipe, openCV, numpy), 아두이노(Servo)  
  
기한 : 11월 30일

## 0.기본 세팅

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## 1.소스 코드

#### 포즈 랜드마크
<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

#### 각도를 계산하는 함수

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

#### 두 점 사이의 거리를 계산하는 함수

In [3]:
def calculate_Distance(landmark1, landmark2):
    a = landmark1
    b = landmark2
    
    dis = (a - b) * 100

    return dis

#### 메인 코드

##### 아두이노 없는 코드

In [11]:
cap = cv2.VideoCapture(0)

state = "error"
elbow_angle_txt = "error"
index_angle_txt = "error"
distance_x_txt = "error"
distance_y_txt = "error"
wrist_distance_txt = "error"

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        cv2.rectangle(image, (0,370), (225,500), (245,117,16), -1)
        cv2.rectangle(image, (400,420), (700,500), (245,117,16), -1)

        try:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_index = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_index = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_elbow_angle = int(left_elbow_angle)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_elbow_angle = int(right_elbow_angle)
            
            cv2.putText(image, str(left_elbow_angle), tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(right_elbow_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            left_index_angle = calculate_angle(left_index, left_wrist, left_elbow)
            left_index_angle = int(left_index_angle)
            right_index_angle = calculate_angle(right_index, right_wrist, right_elbow)
            right_index_angle = int(right_index_angle)
            
            cv2.putText(image, str(left_index_angle), tuple(np.multiply(left_wrist, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(right_index_angle), tuple(np.multiply(right_wrist, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            elbow_distance_x = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)
            elbow_distance_x = int(elbow_distance_x)
            wrist_distance_x = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x)
            wrist_distance_x = int(wrist_distance_x)
            
            cv2.putText(image, str(wrist_distance_x), (600,450), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            
            left_elbow_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y)
            left_elbow_distance_y = int(left_elbow_distance_y)
            left_wrist_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
            left_wrist_distance_y = int(left_wrist_distance_y)
            
            right_elbow_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y)
            right_elbow_distance_y = int(right_elbow_distance_y)
            right_wrist_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
            right_wrist_distance_y = int(right_wrist_distance_y)
            
            if 50 < right_elbow_angle < 100 and 50 < left_elbow_angle < 100:
                elbow_angle_txt = "OK"
            else:
                elbow_angle_txt = "None"
            cv2.putText(image, "elbow angle: " + elbow_angle_txt, (10,390), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if 110 < right_index_angle < 160 and 110 < left_index_angle < 160:
                index_angle_txt = "OK"
            else:
                index_angle_txt ="None"
            cv2.putText(image, "index angle: " + index_angle_txt, (10,420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if elbow_distance_x < wrist_distance_x:
                distance_x_txt = "OK"
            else:
                distance_x_txt = "None"
            cv2.putText(image, "distance X: " + distance_x_txt, (10,450), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if left_elbow_distance_y < left_wrist_distance_y and right_elbow_distance_y < right_wrist_distance_y:
                distance_y_txt = "OK"
            else:
                distance_y_txt = "None"
            cv2.putText(image, "distance Y: " + distance_y_txt, (10,480), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if -15 < wrist_distance_x < -5:
                wrist_distance_txt = "OK"
            else:
                wrist_distance_txt = "None"
            cv2.putText(image, "wrist distance: " + wrist_distance_txt, (410,475), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)           
        except:
            pass
            
        if elbow_angle_txt == "OK" and distance_x_txt == "OK" and distance_x_txt == "OK" and distance_y_txt == "OK" :
            state = "open" 
        elif wrist_distance_txt == "OK" :
            state = "close" 
        else :
            state = "none" 
            
        
        cv2.putText(image, "State: " + state, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(50) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

##### 아두이노 있는 코드

In [4]:
cap = cv2.VideoCapture(0)

arduino = serial.Serial("COM9", 9600)
time.sleep(1)
print("연결됨")

state = "error"
elbow_angle_txt = "error"
index_angle_txt = "error"
distance_x_txt = "error"
distance_y_txt = "error"
wrist_distance_txt = "error"

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        cv2.rectangle(image, (0,370), (225,500), (245,117,16), -1)
        cv2.rectangle(image, (400,420), (700,500), (245,117,16), -1)

        try:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_index = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_index = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_elbow_angle = int(left_elbow_angle)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_elbow_angle = int(right_elbow_angle)
            
            cv2.putText(image, str(left_elbow_angle), tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(right_elbow_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            left_index_angle = calculate_angle(left_index, left_wrist, left_elbow)
            left_index_angle = int(left_index_angle)
            right_index_angle = calculate_angle(right_index, right_wrist, right_elbow)
            right_index_angle = int(right_index_angle)
            
            cv2.putText(image, str(left_index_angle), tuple(np.multiply(left_wrist, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(right_index_angle), tuple(np.multiply(right_wrist, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            elbow_distance_x = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)
            elbow_distance_x = int(elbow_distance_x)
            wrist_distance_x = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x)
            wrist_distance_x = int(wrist_distance_x)
            
            cv2.putText(image, str(wrist_distance_x), (600,450), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            
            left_elbow_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y)
            left_elbow_distance_y = int(left_elbow_distance_y)
            left_wrist_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
            left_wrist_distance_y = int(left_wrist_distance_y)
            
            right_elbow_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y)
            right_elbow_distance_y = int(right_elbow_distance_y)
            right_wrist_distance_y = calculate_Distance(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
            right_wrist_distance_y = int(right_wrist_distance_y)
            
            if 50 < right_elbow_angle < 100 and 50 < left_elbow_angle < 100:
                elbow_angle_txt = "OK"
            else:
                elbow_angle_txt = "None"
            cv2.putText(image, "elbow angle: " + elbow_angle_txt, (10,390), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if 130 < right_index_angle < 155 and 130 < left_index_angle < 155:
                index_angle_txt = "OK"
            else:
                index_angle_txt ="None"
            cv2.putText(image, "index angle: " + index_angle_txt, (10,420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if elbow_distance_x < wrist_distance_x:
                distance_x_txt = "OK"
            else:
                distance_x_txt = "None"
            cv2.putText(image, "distance X: " + distance_x_txt, (10,450), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if left_elbow_distance_y < left_wrist_distance_y and right_elbow_distance_y < right_wrist_distance_y:
                distance_y_txt = "OK"
            else:
                distance_y_txt = "None"
            cv2.putText(image, "distance Y: " + distance_y_txt, (10,480), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            
            if -15 < wrist_distance_x < -5:
                wrist_distance_txt = "OK"
            else:
                wrist_distance_txt = "None"
            cv2.putText(image, "wrist distance: " + wrist_distance_txt, (410,475), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)           
        except:
            pass
            
        if elbow_angle_txt == "OK" and index_angle_txt == "OK" and distance_x_txt == "OK" and distance_x_txt == "OK" and distance_y_txt == "OK" :
            state = "open" 
        elif wrist_distance_txt == "OK" :
            state = "close"
        else :
            state = "none" 
        
        if state == "open":
            var = '1'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)
            
        elif state == "close":
            var = '2'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)
        
        cv2.putText(image, "State: " + state, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('o'):
            var = '1'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)
        
        if cv2.waitKey(10) & 0xFF == ord('p'):
            var = '2'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            var = '3'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)
        
        if cv2.waitKey(10) & 0xFF == ord('w'):
            var = '4'
            var = var.encode('utf-8')
            arduino.write(var)
            time.sleep(0.1)
        
        if cv2.waitKey(10) & 0xFF == ord('h'):
            break

    cap.release()
    cv2.destroyAllWindows()

연결됨
